In [14]:
%%capture
from pathlib import Path

if Path.cwd().stem == "notebooks":
    %cd ..
    %load_ext autoreload
    %autoreload 2

In [15]:
import logging
from pathlib import Path

import holoviews as hv
import hvplot.polars  # noqa
import polars as pl
from polars import col

from src.data.database_manager import DatabaseManager
from src.log_config import configure_logging
from src.plots.correlations import (
    aggregate_correlations_fisher_z,
    calculate_correlations_by_trial,
    plot_correlations_by_participant,
    plot_correlations_by_trial,
    plot_max_correlations_by_participant,
)

logger = logging.getLogger(__name__.rsplit(".", maxsplit=1)[-1])
configure_logging(
    stream_level=logging.DEBUG,
    ignore_libs=["matplotlib", "Comm", "bokeh", "tornado", "param", "numba"],
)

pl.Config.set_tbl_rows(12)  # for the 12 trials
hv.output(widget_location="bottom", size=130)

In [16]:
db = DatabaseManager()
with db:
    df = db.get_table(
        "Merged_and_Labeled_Data",
        exclude_trials_with_measurement_problems=True,
    )

# Rename columns
df = df.rename(
    {
        "rating": "pain_rating",
    }
)

# Remove first 20 seconds
df = df.filter(col("normalized_timestamp") >= 20 * 1000)
df

trial_id,trial_number,participant_id,timestamp,temperature,pain_rating,eda_raw,eda_tonic,eda_phasic,ppg_raw,ppg_ibi_shimmer,heart_rate,heart_rate_exploratory,pupil_l_raw,pupil_r_raw,pupil_r,pupil_l,pupil_r_exploratory,pupil_l_exploratory,pupil,pupil_exploratory,brow_furrow,cheek_raise,mouth_open,upper_lip_raise,nose_wrinkle,normalized_timestamp,stimulus_seed,skin_patch,decreasing_intervals,major_decreasing_intervals,increasing_intervals,strictly_increasing_intervals,strictly_increasing_intervals_without_plateaus,plateau_intervals,prolonged_minima_intervals
u16,u8,u8,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,u16,u8,u16,u16,u16,u16,u16,u16,u16
1,1,1,236939.4492,0.7936,0.5825,29.997654,30.212782,-0.215128,1376.407197,-2.1449,76.965649,77.481282,4.110702,3.926583,3.929316,4.149625,3.929602,4.149863,4.03947,4.039732,0.000009,0.00004,0.026309,7.1032e-8,0.000002,20000.0,243,1,1,0,1,1,0,0,0
1,1,1,237039.4492,0.793556,0.587941,29.981687,30.21038,-0.228693,1360.04548,-2.213915,77.333677,77.94112,4.171716,3.991011,3.929405,4.149136,3.929424,4.149924,4.03927,4.039674,0.000008,0.000039,0.027942,6.5735e-8,0.000002,20100.0,243,1,1,0,0,0,0,0,0
1,1,1,237139.4492,0.793417,0.595235,29.982151,30.208951,-0.2268,1332.860289,1.897561,77.515105,78.091454,4.210685,4.036124,3.929309,4.148921,3.929313,4.149041,4.039115,4.039177,0.000008,0.000039,0.027809,6.0923e-8,0.000002,20200.0,243,1,1,0,0,0,0,0,0
1,1,1,237239.4492,0.793177,0.60058,29.98759,30.207732,-0.220142,1364.225712,-7.180518,77.646147,78.147026,4.208353,4.044555,3.929842,4.149154,3.92984,4.149221,4.039498,4.03953,0.000008,0.00004,0.028769,5.6372e-8,0.000002,20300.0,243,1,1,0,0,0,0,0,0
1,1,1,237339.4492,0.792837,0.602325,29.994336,30.206611,-0.212275,1411.505699,28.381904,77.749248,78.150429,4.163422,4.011142,3.928707,4.149818,3.928705,4.148826,4.039262,4.038765,0.000008,0.000039,0.028358,5.3110e-8,0.000001,20400.0,243,1,1,0,0,0,0,0,0
1,1,1,237439.4492,0.792394,0.613266,29.996586,30.20573,-0.209143,1420.515,45.986135,77.81749,78.13219,4.126558,3.965269,3.939645,4.14824,3.939647,4.149078,4.043943,4.044362,0.000008,0.000039,0.028118,5.0862e-8,0.000001,20500.0,243,1,1,0,0,0,0,0,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
480,12,42,2.8184e6,0.033976,0.0,3.033621,3.094097,-0.000643,1342.086621,-7.417371,71.357566,72.196031,3.443303,4.198385,4.173481,3.500694,4.173038,3.500694,3.837088,3.836866,0.000081,0.000767,0.008733,0.002715,0.002105,179500.0,681,6,2400,1440,0,0,0,0,0
480,12,42,2.8185e6,0.033496,0.0,3.031423,3.094058,-0.000488,1307.163293,-6.579582,71.627735,72.411536,3.456925,4.222612,4.171374,3.461206,4.172201,3.461206,3.81629,3.816704,0.000082,0.000772,0.008841,0.002749,0.002114,179600.0,681,6,2400,1440,0,0,0,0,0


In [17]:
facial_features = [
    "brow_furrow",
    "brow_furrow",
    "cheek_raise",
    "mouth_open",
    "upper_lip_raise",
    "nose_wrinkle",
]


In [18]:
df.filter(participant_id=35, trial_number=7)

trial_id,trial_number,participant_id,timestamp,temperature,pain_rating,eda_raw,eda_tonic,eda_phasic,ppg_raw,ppg_ibi_shimmer,heart_rate,heart_rate_exploratory,pupil_l_raw,pupil_r_raw,pupil_r,pupil_l,pupil_r_exploratory,pupil_l_exploratory,pupil,pupil_exploratory,brow_furrow,cheek_raise,mouth_open,upper_lip_raise,nose_wrinkle,normalized_timestamp,stimulus_seed,skin_patch,decreasing_intervals,major_decreasing_intervals,increasing_intervals,strictly_increasing_intervals,strictly_increasing_intervals_without_plateaus,plateau_intervals,prolonged_minima_intervals
u16,u8,u8,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,u16,u8,u16,u16,u16,u16,u16,u16,u16
397,7,35,1.6289e6,0.501336,0.00625,12.811356,12.773014,0.038342,1305.483929,-2.081084,66.729217,66.163741,3.178047,3.141054,3.099974,3.159852,3.099975,3.159826,3.129913,3.129901,0.000127,0.000287,0.026578,0.000335,0.000396,20000.0,133,6,1981,1189,0,0,0,0,0
397,7,35,1.6290e6,0.494039,0.00625,12.809089,12.764472,0.044617,1265.404004,1.910096,66.608275,66.088916,3.170626,3.136615,3.101029,3.159371,3.101026,3.159383,3.1302,3.130204,0.000128,0.000281,0.025755,0.00033,0.000386,20100.0,133,6,1981,1189,0,0,0,0,0
397,7,35,1.6291e6,0.486768,0.00625,12.806283,12.756595,0.049688,1272.302194,-5.320666,66.528441,66.040257,3.093921,3.06535,3.093032,3.149101,3.093036,3.149102,3.121066,3.121069,0.000132,0.000284,0.026141,0.00032,0.000377,20200.0,133,6,1981,1189,0,0,0,0,0
397,7,35,1.6292e6,0.479546,0.00625,12.805845,12.748423,0.057421,1614.446847,54.811178,66.46143,66.003835,3.079083,3.064105,3.077894,3.127086,3.077889,3.127075,3.10249,3.102482,0.000133,0.000299,0.028053,0.000316,0.000382,20300.0,133,6,1981,1189,0,0,0,0,0
397,7,35,1.6293e6,0.472145,0.00625,12.804182,12.742513,0.061669,1735.594646,53.8289,66.418326,65.987003,2.824795,2.806827,3.055805,3.086286,3.05581,3.086303,3.071045,3.071056,0.000127,0.000302,0.034167,0.000323,0.000368,20400.0,133,6,1981,1189,0,0,0,0,0
397,7,35,1.6294e6,0.464833,0.00625,12.790933,12.732546,0.058387,1461.813768,-12.940733,66.372494,66.008937,2.56413,2.561495,3.032987,3.054236,3.032982,3.054217,3.043611,3.0436,0.000125,0.000307,0.040919,0.000332,0.000365,20500.0,133,6,1981,1189,0,0,0,0,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
397,7,35,1.7884e6,0.159042,0.384976,9.74993,9.860842,-0.004934,1203.986735,-0.894841,80.387014,81.623026,3.18928,3.169855,3.292865,3.39451,3.292866,3.39451,3.343688,3.343688,0.000198,0.000587,0.032952,0.000671,0.000815,179500.0,133,6,1985,0,0,0,0,0,0
397,7,35,1.7885e6,0.157555,0.362854,9.745656,9.860824,-0.005725,1133.769843,-0.916325,80.730848,81.819255,3.214788,3.199034,3.216076,3.306757,3.216073,3.306758,3.261417,3.261415,0.000194,0.000589,0.036377,0.000655,0.000885,179600.0,133,6,1985,0,0,0,0,0,0


# Temperature

In [19]:
COLORS = {
    "temperature_brow_furrow_corr": "red",
    "temperature_cheek_raise_corr": "#2ca02c",
    "temperature_mouth_open_corr": "#d62728",
    "temperature_upper_lip_raise_corr": "#9467bd",
    "temperature_nose_wrinkle_corr": "#ff7f0e",
}

In [20]:
corrs_by_participant = {}
plots = {}

for feature in facial_features:
    col1, col2 = "temperature", feature

    corr_by_trial = calculate_correlations_by_trial(df, col1, col2)
    corrs_by_participant[feature] = aggregate_correlations_fisher_z(
        corr_by_trial, col1, col2, "participant_id", include_ci=True
    )
    plot_correlations_by_trial(corr_by_trial, col1, col2)
    plots[feature] = plot_correlations_by_participant(
        corrs_by_participant[feature], col1, col2, with_config=False, color_map=COLORS
    )

combined_chart = (
    plots["brow_furrow"]
    + plots["cheek_raise"]
    + plots["mouth_open"]
    + plots["upper_lip_raise"]
    + plots["nose_wrinkle"]
)
combined_chart

16:15:27 | DEBUG   | correlations | Removing NaN correlations
16:15:27 | DEBUG   | correlations | Removing NaN correlations
16:15:27 | DEBUG   | correlations | Removing NaN correlations


alt.LayerChart(...)

In [21]:
corrs_by_participant["brow_furrow"].describe()

statistic,participant_id,participant_id_temperature_brow_furrow_corr_mean,participant_id_temperature_brow_furrow_corr_ci_lower,participant_id_temperature_brow_furrow_corr_ci_upper
str,f64,f64,f64,f64
"""count""",42.0,42.0,42.0,42.0
"""null_count""",0.0,0.0,0.0,0.0
"""mean""",21.5,0.056765,-0.063973,0.175702
"""std""",12.267844,0.158167,0.169445,0.160033
"""min""",1.0,-0.20672,-0.377667,-0.10029
"""25%""",11.0,-0.071756,-0.19642,0.049286
"""50%""",22.0,0.063348,-0.052081,0.166031
"""75%""",32.0,0.167485,0.057866,0.287979
"""max""",42.0,0.424,0.310719,0.525404


In [8]:
# Save figure
import os
from pathlib import Path

from dotenv import load_dotenv

load_dotenv()
FIGURE_DIR = Path(os.getenv("FIGURE_DIR"))


# Save the figure
path = FIGURE_DIR / "correlations_with_temperature_face.png"
combined_chart.save(path)
# Or save as SVG for vector graphics:
# combined_chart.save(FIGURE_DIR / "correlations_with_temperature.svg")

# Rating

In [9]:
COLORS = {
    "pain_rating_brow_furrow_corr": "red",
    "pain_rating_cheek_raise_corr": "#2ca02c",
    "pain_rating_mouth_open_corr": "#d62728",
    "pain_rating_upper_lip_raise_corr": "#9467bd",
    "pain_rating_nose_wrinkle_corr": "#ff7f0e",
}

In [10]:
plots = {}

for feature in facial_features:
    col1, col2 = "pain_rating", feature

    corr_by_trial = calculate_correlations_by_trial(df, col1, col2)
    corr_by_participant = aggregate_correlations_fisher_z(
        corr_by_trial, col1, col2, "participant_id", include_ci=True
    )
    plot_correlations_by_trial(corr_by_trial, col1, col2)
    plots[feature] = plot_correlations_by_participant(
        corr_by_participant, col1, col2, with_config=False, color_map=COLORS
    )

combined_chart = (
    plots["brow_furrow"]
    + plots["cheek_raise"]
    + plots["mouth_open"]
    + plots["upper_lip_raise"]
    + plots["nose_wrinkle"]
)
combined_chart

16:12:40 | DEBUG   | correlations | Removing NaN correlations
16:12:41 | DEBUG   | correlations | Removing NaN correlations
16:12:41 | DEBUG   | correlations | Removing NaN correlations


alt.LayerChart(...)

In [11]:
# Save figure
import os
from pathlib import Path

from dotenv import load_dotenv

load_dotenv()
FIGURE_DIR = Path(os.getenv("FIGURE_DIR"))


# Save the figure
path = FIGURE_DIR / "correlations_with_rating_face.png"
combined_chart.save(path)
# Or save as SVG for vector graphics:
# combined_chart.save(FIGURE_DIR / "correlations_with_temperature.svg")